In [1]:
!pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py


[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = "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"

In [3]:
#ollama test

import ollama
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue because of a phenomenon called Rayleigh scattering, which is the scattering of light by small particles or molecules in the atmosphere. The shorter, blue wavelengths are scattered more than the longer, red wavelengths, resulting in the blue color we see.

Here's a more detailed explanation:

1. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.
2. The shorter wavelengths of light, like blue and violet, are scattered more by these small molecules because they interact with them more frequently. This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described the phenomenon in the late 19th century.
3. The longer wavelengths of light, like red and orange, are not scattered as much because they don't interact with the small molecules as often. Instead, they continue to travel in a more direct path t

In [4]:
local_llm = 'llama3'

In [5]:
#Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document
import ollama


In [6]:
urls = ['https://subslikescript.com/movie/The_Lord_of_the_Rings_The_Fellowship_of_the_Ring-120737',
        'https://subslikescript.com/movie/The_Lord_of_the_Rings_The_Two_Towers-167261',
        'https://subslikescript.com/movie/The_Lord_of_the_Rings_The_Return_of_the_King-167260']

In [7]:
docs = [FireCrawlLoader(api_key='5ea20b46-5973-4482-9a50-6f927d066ab1', url=url, mode='scrape').load() for url in urls]

In [8]:
docs[0]


[Document(page_content='[SUBSLIKESCRIPT](/ "Subs like Script - all Movies and TV Shows Transcripts")\n\n[Movies](/movies "All movies transcripts")\n [TV Shows](/series "All Tv Shows transcripts")\n\n*   [Home](/)\n    \n*   [Movies](/movies)\n    \n*   The Lord of the Rings: The Fellowship of the Ring - subtitles like script\n\n Search\n\nThe Lord of the Rings: The Fellowship of the Ring (2001) - full transcript\n==========================================================================\n\nAn ancient Ring thought lost for centuries has been found, and through a strange twist in fate has been given to a small Hobbit named Frodo. When Gandalf discovers the Ring is in fact the One Ring of the Dark Lord Sauron, Frodo must make an epic quest to the Cracks of Doom in order to destroy it! However he does not go alone. He is joined by Gandalf, Legolas the elf, Gimli the Dwarf, Aragorn, Boromir and his three Hobbit friends Merry, Pippin and Samwise. Through mountains, snow, darkness, forests, r

In [9]:
doc_list = [item for sublist in docs for item in sublist]

In [10]:
doc_list

[Document(page_content='[SUBSLIKESCRIPT](/ "Subs like Script - all Movies and TV Shows Transcripts")\n\n[Movies](/movies "All movies transcripts")\n [TV Shows](/series "All Tv Shows transcripts")\n\n*   [Home](/)\n    \n*   [Movies](/movies)\n    \n*   The Lord of the Rings: The Fellowship of the Ring - subtitles like script\n\n Search\n\nThe Lord of the Rings: The Fellowship of the Ring (2001) - full transcript\n==========================================================================\n\nAn ancient Ring thought lost for centuries has been found, and through a strange twist in fate has been given to a small Hobbit named Frodo. When Gandalf discovers the Ring is in fact the One Ring of the Dark Lord Sauron, Frodo must make an epic quest to the Cracks of Doom in order to destroy it! However he does not go alone. He is joined by Gandalf, Legolas the elf, Gimli the Dwarf, Aragorn, Boromir and his three Hobbit friends Merry, Pippin and Samwise. Through mountains, snow, darkness, forests, r

In [11]:
#split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size =250, chunk_overlap = 0)
doc_splits = text_splitter.split_documents(doc_list)

#Filtering out complex meta data and ensuring proper document formatting
filtered_docs = []
for doc in doc_splits:
    #Ensure the doc is an instance of Document and has a metadata attribute
    if isinstance(doc,Document) and hasattr(doc, 'metadata'):
        clean_metadata = {k:v for k,v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_docs.append(Document(page_content=doc.page_content, metadata=clean_metadata))
        


In [12]:
clean_metadata

{'sourceURL': 'https://subslikescript.com/movie/The_Lord_of_the_Rings_The_Return_of_the_King-167260',
 'pageStatusCode': 200,
 'pageError': 'INTERNAL SERVER ERROR'}

In [13]:
filtered_docs

[Document(page_content='[SUBSLIKESCRIPT](/ "Subs like Script - all Movies and TV Shows Transcripts")\n\n[Movies](/movies "All movies transcripts")\n [TV Shows](/series "All Tv Shows transcripts")\n\n*   [Home](/)\n    \n*   [Movies](/movies)\n    \n*   The Lord of the Rings: The Fellowship of the Ring - subtitles like script\n\n Search\n\nThe Lord of the Rings: The Fellowship of the Ring (2001) - full transcript\n==========================================================================', metadata={'sourceURL': 'https://subslikescript.com/movie/The_Lord_of_the_Rings_The_Fellowship_of_the_Ring-120737', 'pageStatusCode': 200}),
 Document(page_content='An ancient Ring thought lost for centuries has been found, and through a strange twist in fate has been given to a small Hobbit named Frodo. When Gandalf discovers the Ring is in fact the One Ring of the Dark Lord Sauron, Frodo must make an epic quest to the Cracks of Doom in order to destroy it! However he does not go alone. He is joined b

In [14]:
#save the filtered doc list as a json for future use

import json
def document_to_dict(doc):
    return {
        "page_content": doc.page_content,
        "metadata": doc.metadata
    }
    
#Convert document object to dictionaries
doc_dicts = [document_to_dict(doc)for doc in filtered_docs]

#Saving to a json file
with open("lotr_scraped_doc.json", "w", encoding="utf-8") as f:
    json.dump(doc_dicts, f, ensure_ascii=False, indent=2)

In [15]:
#Loading the json
with open("lotr_scraped_doc.json", "r", encoding="utf-8") as f:
    loaded_dicts = json.load(f)

filtered_docs = [Document(**doc_dict) for doc_dict in loaded_dicts]

In [16]:
#Add to vectorDB

vectorstore = Chroma.from_documents(
    documents = filtered_docs,
    collection_name="rag-chroma",
    embedding=GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf"),
)

retriever = vectorstore.as_retriever()

In [17]:
!ollama list

NAME                    	ID          	SIZE  	MODIFIED       
llama3:latest           	365c0bd3c000	4.7 GB	16 minutes ago	
mxbai-embed-large:latest	468836162de7	669 MB	10 days ago   	
mistral:latest          	2ae6f6dd7a3d	4.1 GB	10 days ago   	


In [24]:
#Retrievar Grader
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.llms.ollama import Ollama

#llm 
llm = ChatOllama(model = local_llm, format='json', temperature=0)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance
of a retrieved document to a user question. If the document contains keywords related to the user question,
grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
Provide the binary score as a JSON with a single key 'score' and no preamble or explaination.
<|eot_id|><|start_header_id|>user<|end_header_id|>
Here is the retrieved document: \n\n {document} \n\n
Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "Why should we do business management?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))


{'score': 'no'}


In [21]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3", base_url="http://localhost:11434")
llm.invoke("Why is the sky blue?")

"What a great question!\n\nThe sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. Here's what happens:\n\n1. **Light from the sun**: When sunlight enters Earth's atmosphere, it contains all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).\n2. **Interaction with molecules**: As light travels through the air, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.\n3. **Scattering**: The smaller molecules scatter the shorter wavelengths of light, such as blue and violet, more efficiently than the longer wavelengths, like red and orange. This is known as Rayleigh scattering.\n4. **Blue dominance**: As a result of this scattering, the blue and violet light becomes dispersed throughout the atmosphere, reaching our eyes from all directions. The shorter wavelength of blue light makes it more easily scatter

In [102]:
#Generating and answer
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
Question: {question}
Context: {document}
Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

llm = ChatOllama(model=local_llm, temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = prompt | llm | StrOutputParser()

question = "What was the main mission of Frodo?"
docs = retriever.invoke(question)
generated_answer = rag_chain.invoke({"document": docs, "question": question})
print(generated_answer)



The main mission of Frodo was to destroy the One Ring by taking it to Mount Doom and throwing it into the fiery depths.


## Websearch using Tavily

In [33]:
os.environ['TAVILY_API_KEY'] = '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'

from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(max_results=3)

In [111]:
### Hallucination Grader

# LLM
llm_halu = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt_halu = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
an answer is grounded in / supported by a set of facts. Give a binary score 'yes' or 'no' score to indicate 
whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
Here are the facts:
\n----------\n
{documents}
\n----------\n
Here is the answer: {generated} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generated", "documents"],
)

hallucination_grader = prompt_halu | llm_halu | JsonOutputParser()

hallucination_grader.invoke({"documents": docs, "generated": generated_answer})


{'score': 'yes'}

In [112]:
### Answer Grader

# LLM
llm_ans = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt_ans = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation. 
<|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
{generated}
\n----------\n
Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generated", "question"],
)

answer_grader = prompt_ans | llm_ans | JsonOutputParser()
answer_grader.invoke({"question": question, "generated": generated_answer})


{'score': 'yes'}

In [113]:
from typing_extensions import TypedDict
from typing import List

##state
class GraphState(TypedDict):
    """
    Represents state of the graph
    
    Attributes:
    question: question
    generation: LLM generation
    web_search: whether to add search
    documents: list of documents
    
    """
    
    question : str
    generation : str
    web_search: str
    documents: List[str]
    
from langchain.schema import Document
    
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]
    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search
    
    Args:
        state (dict): The current graph state
    
    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state['question']
    documents = state['documents']
    
    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score['score']
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web_search
            web_search='Yes'
            continue
    return {"documents":filtered_docs, "question": question, "web_search": web_search}

def generate(state):
    """
    Generate answer using RAG on retrived documents
    Args:
        state (dict): The current graph state
    
    Returns:
        state (dict): New key added to state , generated_answer, that contains the LLM generation
        
    """
    print("----GENERATE-----")
    question = state['question']
    documents = state['documents']
    
    #RAG generation
    generated_answer = rag_chain.invoke({"document":documents, "question":question})
    return {"documents": documents, "question":question, "generated": generated_answer}

def web_search(state):
    
    """
    Web search based  on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """
    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

#Conditional edge

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"
    
    
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generated answer in grounded to facts from the documents or not

    Args:
        state (dict): The current graph state

    Returns:
        str: Decition for next node to call
    """
    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generated = state["generated"]
    hallucination_score = hallucination_grader.invoke({"documents": documents, "generated": generated})
    hallucination_grade = hallucination_score['score']

    # Check hallucination
    if hallucination_grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        answer_score = answer_grader.invoke({"question": question, "generated": generated})
        answer_grade = answer_score['score']
        if answer_grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)
# Define the nodes
workflow.add_node("websearch", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generate


In [114]:

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)   

In [115]:
#app
app = workflow.compile()

#test
from pprint import pprint
inputs = {"question":"What was the main mission assigned to Frodo?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finizhed running: {key}:")
print(value["generated"])


---RETRIEVE---
'Finizhed running: retrieve:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
'Finizhed running: grade_documents:'
---WEB SEARCH---
'Finizhed running: websearch:'
----GENERATE-----
---CHECK HALLUCINATIONS---


KeyError: 'score'